Welcome to the TrekBot Colab notebook! This notebook behaves exactly like a jupyter notebook. Let's dive right in!

First step: put the JSON data file here inside a folder called "data":
https://www.kaggle.com/datasets/gjbroughton/start-trek-scripts?resource=download

Now we'll install a few necessary packages.

In [1]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.1 MB/s eta 0:00:00


Next we import the packages we'll need.


**pandas** is a package that allows us to conveniently store and manipulate data in a data structure known as a Dataframe. (This is similar to a Dataframe in R, for those familiar with R.) It’s a very common tool for anyone doing data science in python.

**sklearn** is the package formally called “scikit-learn”, and contains a wide range of statistical and machine learning methods. It’s another very common package for data scientists in python.

**numpy** is python’s main numeric library, and allows us to do things like work with arrays, matrices, dot products, etc.

**json** is a package for interacting with json files. Our data is formatted as a single json file, so this is useful for us here.

**os** helps us with file management and command-line commands.

**openai** is a package containing functions that allow us to easily make API calls to OpenAI’s models in python.

Finally, we import **cosine_similarity** from sklearn, since it’s a specialized function that we need today.

In [2]:
import pandas as pd
import numpy as np
import json
import openai
from sklearn.metrics.pairwise import cosine_similarity
import os

CHUNK_SIZE = 600
OVERLAP = 20

Remember the OpenAI API key you created? Copy and paste it in the cell below.

In [ ]:
openai.api_key = input("Paste your OpenAI API key here and hit enter:");

Here's what the model is doing: we have a long piece of text that we want ChatGPT to be able to answer questions about. We first break that text up into chunks containing 600 words (technically called “tokens”), where each chunk overlaps 20 words with the following chunk. We then send these chunks to OpenAI to obtain their embeddings. When we ask a question about our text, we find the question’s embedding, and use cosine similarity to find the chunk of text that is closest to our question. We then send a query to ChatGPT that includes our original question, as well as the chunk of text as context.

We loop over all the chunks, and send each one to OpenAI, get back the embedding, and then write a new line to the Dataframe df. Note that we are casting the embedding response (a string) to a numpy array. We do this because we will be doing numerical operations on the embedding in just a moment.

In [4]:
scripts = json.load(open("adeola_custom_text/adeola_sample_text.json", encoding='ascii'))
text = scripts['Vusi_writes']['LinkedIn_post']
text_list = text.split()
chunks = [text_list[i:i+CHUNK_SIZE] for i in range(0, len(text_list), CHUNK_SIZE-OVERLAP)]
df = pd.DataFrame(columns=['chunk', 'gpt_raw', 'embedding'])
for chunk in chunks:
    f = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=" ".join(chunk),
    )
    df.loc[len(df.index)] = (chunk, f, np.array(f['data'][0]['embedding']))

In [5]:
df.head()

,chunk,gpt_raw,embedding
0,"[AI, Robot, as, CEO:, Company, Shatters, Stock...","{'object': 'list', 'data': [{'object': 'embedd...","[-0.00732967397198081, -0.045327551662921906, ..."


Now, let’s define our query and get its embedding. Our query is a simple question: who was the captain of the Excalibur? A bit of context: in this episode, a small detail is that one of the crew members was assigned to command a ship for this one episode only, and it’s a minor detail in the plot of the episode. In fact, if you ask ChatGPT this question without giving it the script, it doesn’t know the answer. We’ll see that with the right chunk of text, identified by cosine similarity, ChatGPT can answer correctly.

We calculate the cosine distance from our query to each chunk, and save the chunk that is most similar to a variable called context_chunk.

Finally, we assemble the full query, including the chunk we identified, and send it to ChatGPT via the API:

In [8]:
query = "Tang Yu, an AI robot, was appointed as CEO of which company?"
f = openai.Embedding.create(
    model="text-embedding-ada-002",
    input=query
)
query_embedding = np.array(f['data'][0]['embedding'])

similarity = []
for arr in df['embedding'].values:
    similarity.extend(cosine_similarity(query_embedding.reshape(1, -1), arr.reshape(1, -1)))
context_chunk = chunks[np.argmax(similarity)]

query_to_send = "CONTEXT: " + " ".join(context_chunk) + "\n\n" + query
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= query_to_send,
  max_tokens=100,
  temperature=0
)

In [10]:
print(query_to_send)

CONTEXT: AI Robot as CEO: Company Shatters Stock Market Records with Unprecedented Profits. In a ground-breaking move, a gaming company in Hong Kong appointed an AI robot as its CEO, signalling the growing potential of artificial intelligence to assume human roles. This decision took place even before the widespread recognition of AI chatbots like ChatGPT. Remarkably, this innovative move led to the company achieving unprecedented profits in the stock market. In an awe-inspiring move, Tang Yu, an AI robot, was appointed as CEO of NetDragon Websoft. The business not only achieved unparalleled success in Hong Kong's stock market but also witnessed an astonishing surge in its stock value. TheAI chatbot was recognised by the industry as a pioneer in implementing AI to improve operational productivity and change corporate administration. Among Tang Yu's responsibilities at NetDragon Websoft were examining high-level statistics, evaluating risks, and nurturing an effective workplace. As the 

Let's test our bot. Did it get it right? Execute the cell below to find out!

In [11]:
print(response['choices'][0]['text'].strip())

Tang Yu was appointed as CEO of NetDragon Websoft, a gaming company in Hong Kong.
